In [1]:
import nyroglancer
import h5py
import os
import numpy as np
from scipy.stats import logistic
from scipy.special import logit, expit

pure=True

combined = '/groups/saalfeld/home/funkej/nils/combined_predictions/sample_C_padded_20160501.aligned.filled.cropped.r11.100000.c100.62:153.0:268.0:268.hdf'
f = h5py.File(combined, 'r')

mean_pred_affinities = f['volumes/labels/mean_pred_affinities'].value
mean_entropy = f['volumes/labels/mean_entropy']
raw = f['volumes/raw']
if not pure:
    mean_aleatoric = f['volumes/labels/mean_aleatoric'].value
    mean_aleatoric = abs(expit(logit(mean_pred_affinities) + mean_aleatoric) - expit(logit(mean_pred_affinities) - mean_aleatoric)) 
pred_affinities_0 = f['volumes/labels/0_pred_affinities']
var_affs = f['volumes/labels/var_pred_affinities']

raw = np.array(raw, dtype=np.float32)*0.5 + 0.5

voxel_size=(4,4,40)
f2 = h5py.File('./predictions/140000/p_0/sample_C_padded_20160501.aligned.filled.cropped.62:153.hdf')
#aff_offset = tuple([off for off in f2["volumes/labels/pred_affinities"].attrs.items()[0][1][::-1]])
#raw_offset = tuple([off for off in f2["volumes/raw"].attrs.items()[0][1][::-1]])
aff_offset = ([0,0,0])
raw_offset = ([0,0,0])

shader = """void main() { 
  vec3 color = vec3(toNormalized(getDataValue(0)),
                    toNormalized(getDataValue(1)),
                    toNormalized(getDataValue(2)));
  if ((color[2] + color[1] + color[0])/3. < 0.5) 
  emitRGB(vec3(1,0,0));
  else (emitRGB(vec3(1,1,1)));}"""

viewer = nyroglancer.Viewer()
viewer.add(raw, name='raw', voxel_size=voxel_size, offset=raw_offset)
viewer.add(mean_pred_affinities, name='affs_mean', voxel_size=voxel_size, shader=nyroglancer.shaders.rgb(), offset=aff_offset)
viewer.add(pred_affinities_0, name='affs_0', voxel_size=voxel_size, shader=nyroglancer.shaders.rgb(), offset=aff_offset)
if not pure:
    viewer.add(mean_aleatoric, name='aleatoric_mean', voxel_size=voxel_size, shader=nyroglancer.shaders.rgb(), offset=aff_offset)
viewer.add(mean_entropy, name='entropy_mean', voxel_size=voxel_size, shader=nyroglancer.shaders.rgb(), offset=aff_offset)
viewer.add(var_affs, name='var_affs', voxel_size=voxel_size, shader=nyroglancer.shaders.rgb(), offset=aff_offset)
viewer.set_hostname('localhost:8880')
viewer.show()

### 